<a href="https://colab.research.google.com/github/mancinimassimiliano/DeepLearningLab/blob/master/Lab3/domain_adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Unsupervised Domain Adaption.

### Domain Alignment Layers for Unsupervised Domain Adaptation.

We have at disposal a source dataset (images with labels) and a target dataset (images *without* labels). Our goal is to train a classifier on the source and target dataset that can perform well on the target test dataset. This problem setting is known as **Unsupervised Domain Adaptation**.

In this case the training set consists of the following:


*   Source dataset: SVHN ~ 73k *labeled* data.
*   Target dataset> MNIST ~ 60k *unlabeled* data, eaning no associated labels.

Test set contains 10k MNIST test samples, which are used for evaluation.

We will use Batch Norm based Domain Alignment (DA) Layers for performing domain adaptation.



In [0]:
# import libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F

import os

### Define the network with Domain Alignment Layers.

Each DA layer consist of domain specific Batch Normalization Layers followed by domain agnostic scale-shift operation. Note that the domain specific BN layer will accumulate the domain specific first and second order statistics, i.e., mean and std. It can achieved by setting `track_running_stats=True`.

Details for BN can be found [here](https://pytorch.org/docs/stable/nn.html#batchnorm2d).

### Network architecture adopted from [here](http://proceedings.mlr.press/v37/ganin15-supp.pdf)
{Conv(3, 64, 5, 2) -> DA -> ReLU } ->MaxPool(3,2) -> {Conv(64, 64, 5, 2) -> DA -> ReLU } -> MaxPool(3,2) -> {Conv(64, 128, 5, 2) -> DA -> ReLU } -> {Linear(6272, 3072) -> DA -> ReLU -> Dropout} ->  {Linear(3072, 2048) -> DA -> ReLU -> Dropout} -> {Linear(2048, 10) -> DA}

In [0]:
class DIALNet(nn.Module):
	def __init__(self):
		super(DIALNet, self).__init__()
		# TODO layers

	def forward(self, x):

		if self.training:
			# TODO conv
			# TODO split source and target batches
			# TODO DA layer
      # TODO Relu
      # TODO Pooling

			# TODO conv
			# TODO split source and target batches
			# TODO DA layer
      # TODO Relu
      # TODO Pooling

			# TODO conv
			# TODO split source and target batches
			# TODO DA layer
      # TODO Relu

			x = x.view(x.shape[0], -1)
			# TODO Linear
			# TODO split source and target batches
			# TODO DA layer
      # TODO Relu
      # TODO Dropout

			# TODO Linear
			# TODO split source and target batches
			# TODO DA layer
      # TODO Relu
      # TODO Dropout
      
			# TODO Linear
			# TODO split source and target batches
			# TODO DA layer
      
		else: # only for the target domain
			# TODO conv
			# TODO DA layer
      # TODO Relu
      # TODO Pooling

			# TODO conv
			# TODO DA layer
      # TODO Relu
      # TODO Pooling

			# TODO conv
			# TODO DA layer
      # TODO Relu

			x = x.view(x.shape[0], -1)
			# TODO Linear
			# TODO DA layer
      # TODO Relu
      # TODO Dropout

			# TODO Linear
			# TODO DA layer
      # TODO Relu
      # TODO Dropout
      
			# TODO Linear
			# TODO DA layer
		return x

### Define cross-entropy loss function

For the labeled source data we can compute the cross entropy loss

$L^{s}(\theta) = - \frac{1}{m} \displaystyle \sum^{m}_{i=1} y_{i}\log{p_i}$

where $p_i$ is the output from the network for the $i^{th}$ input and $y_i$ is the corresponding ground truth.

In [0]:
def get_ce_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

### Define Entropy loss function

For the unlabeled target samples we can no longer compute the cross entropy loss because the labels are not available. Instead we will compute the Entropy loss, which is defined as follows:

$L^{t}(\theta) = - \frac{1}{m} \displaystyle \sum^{m}_{i=1} p_{i}\log{p_i}$

where $p_i$ is the output from the network for the $i^{th}$ input.

For more details read [this](https://arxiv.org/pdf/1704.08082.pdf).

In [0]:
def get_entropy_loss(x):
  p = F.softmax(x, dim=1)
  q = F.log_softmax(x, dim=1)
  b = p * q
  b = -1.0 * b.sum(-1).mean()
  return b

### Define the optimizer

In [0]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

### Test function

In [0]:
def test(net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  # Strictly needed if network contains layers which has different behaviours between train and test
  net.eval()
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

### Train function

In [0]:
def train(net, source_data_loader, target_data_loader, optimizer, 
          get_ce_cost_function, entropy_loss_weight, device='cuda:0'):
  source_samples = 0.
  target_samples = 0.
  cumulative_ce_loss = 0.
  cumulative_en_loss = 0.
  cumulative_accuracy = 0.
  
  # TODO target iterator

  # Strictly needed if network contains layers which has different behaviours between train and test
  net.train()
  for batch_idx, (inputs_source, targets) in enumerate(source_data_loader):
    
    try:
      # TODO get the target batch
    except:
      # TODO target iterator
      # TODO get the target batch
    
    # TODO concat the source and target batches
    
    # Load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
      
    # Forward pass
    outputs = net(inputs)
    
    # split the source and target outputs
    # TODO
    
    # Apply the losses
    # TODO cross entropy loss
    # TODO entropy loss
    
    # TODO weighted sum of the losses
    
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Reset the optimizer
    optimizer.zero_grad()

    # Better print something, no?
    source_samples+=inputs_source.shape[0]
    target_samples+=inputs_target.shape[0]
    
    cumulative_ce_loss += ce_loss.item()
    cumulative_en_loss += en_loss.item()
    _, predicted = source_output.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_ce_loss/source_samples, cumulative_en_loss/target_samples, cumulative_accuracy/source_samples*100

### Define the function that fetches a data loader that is then used during iterative training.

In [0]:
def get_data(batch_size, test_batch_size=256):
  
  # Prepare data transformations and then combine them sequentially
  transform_mnist = list()
  transform_mnist.append(T.ToTensor())                                              # converts Numpy to Pytorch Tensor
  transform_mnist.append(T.Lambda(lambda x: F.pad(x, (2, 2, 2, 2), 'constant', 0))) # pad zeros to make MNIST 32 x 32
  transform_mnist.append(T.Lambda(lambda x: x.repeat(3, 1, 1)))                     # to make MNIST RGB instead of grayscale
  transform_mnist.append(T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]))    # Normalizes the Tensors between [-1, 1]
  transform_mnist = T.Compose(transform_mnist)                                      # Composes the above transformations into one.
  
  transform_svhn = list()
  transform_svhn.append(T.ToTensor())                                              # converts Numpy to Pytorch Tensor
  transform_svhn.append(T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]))    # Normalizes the Tensors between [-1, 1]
  transform_svhn = T.Compose(transform_svhn)                                       # Composes the above transformations into one.
  
  # Prepare datasets
  
  # Load SVHN
  source_training_data = torchvision.datasets.SVHN('./data/svhn', split='train', transform=transform_svhn, download=True) 
  
  # Load MNIST
  target_training_data = torchvision.datasets.MNIST('./data/mnist', train=True, transform=transform_mnist, download=True) 
  target_test_data = torchvision.datasets.MNIST('./data/mnist', train=False, transform=transform_mnist, download=True)
  
  # Initialize dataloaders
  source_train_loader = torch.utils.data.DataLoader(source_training_data, batch_size, shuffle=True, drop_last=True)
  target_train_loader = torch.utils.data.DataLoader(target_training_data, batch_size, shuffle=True, drop_last=True)
  
  target_test_loader = torch.utils.data.DataLoader(target_test_data, test_batch_size, shuffle=False)
  
  return source_train_loader, target_train_loader, target_test_loader

### Wrapping everything up

Finally, we need a main function which initializes everything + the needed hyperparameters and loops over multiple epochs (printing the results).

In [0]:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
'''

def main(batch_size=32, 
         device='cuda:0', 
         learning_rate=0.01, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=50,
         entropy_loss_weight=0.1):
  
  source_train_loader, target_train_loader, target_test_loader = get_data(batch_size)
  
  net = DIALNet().to(device)
  
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  cost_function = get_ce_cost_function()
  
  for e in range(epochs):
    train_ce_loss, train_en_loss, train_accuracy = train(net=net, source_data_loader=source_train_loader, 
                                                         target_data_loader=target_train_loader, 
                                                         optimizer=optimizer, get_ce_cost_function=cost_function,
                                                         entropy_loss_weight=entropy_loss_weight)
    test_loss, test_accuracy = test(net, target_test_loader, cost_function)
    
    print('Epoch: {:d}'.format(e+1))
    print('\t Train: CE loss {:.5f}, Entropy loss {:.5f}, Accuracy {:.2f}'.format(train_ce_loss, train_en_loss, train_accuracy))
    print('\t Test: CE loss {:.5f}, Accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')

Let's train!

In [0]:
main()